<a href="https://colab.research.google.com/github/hmyrcmn/FaceMaskDetectionYolo/blob/main/HairnetDetectionsBigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Kaggle API anahtarını yükleyin
from google.colab import files
files.upload()

# 2. API anahtarını taşıyın ve gizli bir şekilde saklayın
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 3. Veri setini indirin
!kaggle datasets download -d kierangriffin/hairnetbig

# 4. Veri setini çıkartın
!unzip -q hairnetbig.zip -d ./hairnetbig-detection


Saving kaggle.json to kaggle.json
100% 4.12G/4.12G [00:47<00:00, 73.5MB/s]
100% 4.12G/4.12G [00:47<00:00, 93.8MB/s]


In [2]:
# An image in the Data Set(Veri Seti içerisinde bir resim)
Image.open("/content/hairnetbig-detection/data/strands00002_00029_10000_v0.png")


NameError: ignored

In [ ]:
import cv2
import numpy as np
import torch
from IPython.display import Image, display


In [ ]:
!pip install ultralytics

import torch

# Modeli GitHub'dan çekin
model_url = "https://github.com/hmyrcmn/FaceMaskDetectionYolo/raw/main/yolov8n.pt"
model_path = "yolov8n.pt"
torch.hub.download_url_to_file(model_url, model_path)

# Modeli yükle
model = torch.load(model_path)

In [ ]:
test_image_path = "/content/hairnetbig-detection/data/strands00002_00004_10000_v0.png"  # Test için bir resim seçin
# Resmi okuyun
image = cv2.imread(test_image_path)

# Resmi YOLOv8 modeliyle işleyin
results = model(image)

# Sonuçları görselleştirin
for result in results.pred[0]:
    label = result[-1]
    confidence = result[4]
    if confidence > 0.5:  # Belirli bir güven eşiği belirleyin
        x1, y1, x2, y2 = map(int, result[:4])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sonuçları görselleştirin
display(Image(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))


In [ ]:
# Resmi okuyun
image = cv2.imread(test_image_path)

# Resmi YOLOv8 modeliyle işleyin
results = model(image)

# Sonuçları görselleştirin
for result in results.pred[0]:
    label = result[-1]
    confidence = result[4]
    if confidence > 0.5:  # Belirli bir güven eşiği belirleyin
        x1, y1, x2, y2 = map(int, result[:4])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Sonuçları görselleştirin
display(Image(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))


In [ ]:
# Gerekli kütüphaneleri içe aktarın
import numpy as np

# Dosyanın yolunu belirtin
file_path = '/content/hairnetbig-detection/convdata/strands00002_00034_10000.convdata'

# Dosyayı açın ve verileri okuyun
with open(file_path, 'rb') as file:
    # Verileri okuyun
    data = file.read()

# Veriyi işleyin
# Örnek olarak, verinin ilk 10 baytını görüntüleyelim
print(data[:10])


In [ ]:
# I import libraries that I will use.(Kullanacağım kütüphaneleri import ediyorum.)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For preparing (Hazırlık için)
import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

In [ ]:
# Tagged data in XML format(Etiketlenmiş verilerin XML formatındaki hali)
with open('/content/face-mask-detection/annotations/maksssksksss0.xml') as f:
    contents = f.read()
    print(contents)

In [ ]:
# Converting XML format data to TXT format(XML formatındaki verileri TXT formatına dönüştürme)
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2

    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

classes = ["without_hairnet", "with_hairnet", "hairnet_weared_incorrect"]

input_dir = "/content/face-mask-detection/annotations"
output_dir = "/content/labels"
image_dir = "/content/face-mask-detection/images"


if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

import glob

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

    # Parse the content of the xml file(Xml dosyasının içeriğini ayrıştırın)
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text

        # check for new classes and append to list(yeni sınıfları kontrol edin ve listeye ekleyin)
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)

        # convert data to string(verileri dizeye dönüştür)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a yolo format text file for each xml file(her xml dosyası için yolo formatında bir metin dosyası oluşturun)
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))
# generate the classes file as reference(sınıf dosyasını referans olarak oluştur)
with open("/content/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

In [ ]:
# An image in the Data Set(Veri Seti içerisinde bir resim)
Image.open("/content/face-mask-detection/images/maksssksksss0.png")
